# Aquisição e Manipulação de Dados

## Modelagem do Futebol Brasileiro

### Autores: Hanna Rodrigues e Igor Michels

_________________________

### Import's

In [1]:
from functions import *
from glob import glob
import numpy as np
import matplotlib.dates as dates

_________________________

### Objetivos do negócio:

O projeto objetiva a criação de um modelo de predição de resultados gerais dos jogos do futebol brasileiro em vitória, derrota ou empate, bem como a inferência do campeão brasileiro utilizando um modelo que será implementado utilizando dados disponíveis sobre jogos da Série A do Brasileirão que podem ser encontrados no site da CBF e em outros sites esportivos.


### Avaliação da situação atual:

Atualmente casas de apostas possuem bons modelos preditivos para a elaboração das odds, enquanto modelos mais simples são utilizados, seja em pesquisa ou seja com objetivos didáticos. Além disso, existem muitos dados disponíveis acerca do tema, uma vez que o futebol é o esporte mais popular do planeta, ou seja, pode-se encontrar os mais variados dados facilmente.


### Objetivo da mineração de dados:

A coleta de dados objetiva, inicialmente, os dados mais gerais de uma partida:

 + mandante;
 + visitante;
 + placar e;
 + data.

Tais dados podem ser obtidos diretamente do site da CBF, como realizado [nesse projeto](https://github.com/IgorMichels/Brazilian_Soccer).


### Plano do projeto:
    
 + Coletar dados do futebol brasileiro via scraping
     + ver [esse repositório](https://github.com/IgorMichels/Brazilian_Soccer/tree/main/Scrape_and_some_views/All%20data/Serie%20A)
 + Explorar e verificar a qualidade dos dados coletados
 + Selecionar, limpar e reformatar os dados que alimentarão o modelo
 + Construir e treinar o modelo de previsão de partidas
     + elaborar modelo estimando forças para cada clube, utilizando os jogos anteriores para realizar o treinamento, com jogos mais recentes tendo peso maior
 + Validar e analisar as previsões do modelo
     + para essa etapa a ideia é utilizar os dados do começo dos campeonatos de 2019 e 2020 (10 primeiras rodadas ou primeiro turno inteiro) para realizar o treinamento do modelo e utilizar os parâmetros obtidos para validar o modelo nesses anos
 + Inferir o campeão brasileiro com o modelo implementado
 + Revisar e fazer o deploy do projeto


## II. Data Understanding  (apagar célula)

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

# Entendimento dos Dados

### Coleta Inicial dos Dados

A coleta inicial dos dados já foi realizada em um projeto anterior, conforme seção acima. Dessa forma, realizamos apenas o download dos dados para esse projeto, considerando todos os jogos de todas as competições da CBF no período de 2013 a 2020.

### Descrição dos Dados

Os dados coletados estão no formato ```csv```, com a súmula crua e completa de cada partida, conforme pode ser visto abaixo.

In [2]:
game = 'Data/Serie A/2020/Game 001.csv' # Flamengo e Atlético-MG pela primeira rodada de 2020
lines_2_print = 25
lines = 0
with open(game) as file:
    reader = csv.reader(file, delimiter = ',')
    for row in reader:
        print(row)
        
        lines += 1
        if lines == lines_2_print:
            break

['Campeonato:', 'Campeonato Brasileiro - Série A / 2020', 'Rodada:', '1', '', '']
['Jogo:', 'Flamengo / RJ X Atlético / MG', '', '', '', '']
['Data:', '09/08/2020', 'Horário:', '16:00', 'Estádio:', 'Jornalista Mário Filho / Rio de Janeiro']
['Arbitragem', '']
['Arbitro:', 'Raphael Claus (FIFA / SP)']
['Arbitro Assistente 1:', 'Daniel Paulo Ziolli (AB / SP)']
['Arbitro Assistente 2:', 'Anderson José de Moraes Coelho (AB / SP)']
['Quarto Arbitro:', 'Rafael Martins de Sá (CD / RJ)']
['Analista de Campo:', 'Edilson Soares da Silva (CBF / RJ)']
['VAR:', 'Thiago Duarte Peixoto (AB / SP)']
['AVAR1:', 'Vinicius Furlan (AB / SP)']
['AVAR2:', 'Enderson Emanoel Turbiani da Silva (AB / SP)']
['Observador de VAR:', 'Nilson de Souza Monção (CBF / RJ)']
['Quality manager:', 'Mikael Silva de Araujo (CBF / RJ)']
['Cronologia', '', '', '', '', '', '', '']
['1o Tempo', '2o Tempo', '', '', '', '', '', '']
['Entrada do mandante:', '15:50', 'Atraso:', 'Não Houve', 'Entrada do mandante:', '16:57', 'Atraso:',

Os dados da súmula foram retirados de um ```PDF``` e convertidos para ```csv``` por meio da biblioteca ```tabula```.

### Exploração dos Dados

Essa etapa pensamos em adiar um pouco para realizar após a limpeza dos dados, uma vez que os mesmos precisam ser tratados. Mas uma análise rápida dos dados nos mostra que todos os dados de nosso interesse se encontram no ínicio da súmula, antes da relação de jogadores.

Podemos ver que até esse ponto já pudemos coletar informações como os clubes que estão se enfrentando, a data da partida e o resultado final do jogo:

In [3]:
with open(game) as file:
    reader = csv.reader(file, delimiter = ',')
    for row in reader:
        if 'Relação de Jogadores' in row:
            break
            
        print(row)

['Campeonato:', 'Campeonato Brasileiro - Série A / 2020', 'Rodada:', '1', '', '']
['Jogo:', 'Flamengo / RJ X Atlético / MG', '', '', '', '']
['Data:', '09/08/2020', 'Horário:', '16:00', 'Estádio:', 'Jornalista Mário Filho / Rio de Janeiro']
['Arbitragem', '']
['Arbitro:', 'Raphael Claus (FIFA / SP)']
['Arbitro Assistente 1:', 'Daniel Paulo Ziolli (AB / SP)']
['Arbitro Assistente 2:', 'Anderson José de Moraes Coelho (AB / SP)']
['Quarto Arbitro:', 'Rafael Martins de Sá (CD / RJ)']
['Analista de Campo:', 'Edilson Soares da Silva (CBF / RJ)']
['VAR:', 'Thiago Duarte Peixoto (AB / SP)']
['AVAR1:', 'Vinicius Furlan (AB / SP)']
['AVAR2:', 'Enderson Emanoel Turbiani da Silva (AB / SP)']
['Observador de VAR:', 'Nilson de Souza Monção (CBF / RJ)']
['Quality manager:', 'Mikael Silva de Araujo (CBF / RJ)']
['Cronologia', '', '', '', '', '', '', '']
['1o Tempo', '2o Tempo', '', '', '', '', '', '']
['Entrada do mandante:', '15:50', 'Atraso:', 'Não Houve', 'Entrada do mandante:', '16:57', 'Atraso:',

### Verificando a Qualidade dos Dados

Essa etapa acaba sendo rápida nesse caso, pois os dados foram obtidos das súmulas das partidas que estão no site da CBF, ou seja, são dados oficiais de cada jogo e que se encontram no site oficial do organizador do evento.

## III. Data Preparation (apagar célula)

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

# Preparação dos Dados

### Seleção e Limpeza dos Dados

Conforme citado anteriormente, tomamos os dados de todos os jogos de todas as competições da CBF de 2013 a 2020. Como estamos interessados na modelagem da primeira divisão, nessa etapa iremos fazer a seleção dos jogos entre os clubes que já estiveram na Série A, em qualquer que seja a competição.

Pensando em otimizar o processo, decidimos unificar a etapa de seleção e limpeza dos dados. Dessa forma, enquanto passamos por cada jogo, já iremos ver se os clubes que estão se enfrentando estiveram na Série A em algum desses anos. Se sim, então a gente guarda os dados dessa partida, caso contrário essa partida não será de grande valia para nossa análise, então descartamos a mesma.

**OBS:** A ideia de pegar todas as competições se dá pelo fato de que, na Série A, ocorre, a cada ano, a renovação de 20% dos clubes (4 são rebaixados e 4 são promovidos), então para termos dados prévios desses clubes que foram promovidos uma opção é a de utilizar os jogos de divisões inferiores.

In [4]:
# our code here
save = []
competitions = sorted(glob('Data/S*')) + glob('Data/C*') # garantir que a Série A seja a primeira da lista
competitions1 = [competitions[0]] # only Serie A
competitions2 = competitions[1:]

clubs = []
for competition in competitions1:
    years = sorted(glob(competition + '/*'))
    for year in years:
        games = sorted(glob(year + '/Game *'))
        for game in games:
            save.append([game[5:game.find('/', 5)]])
            try:
                teams = find_teams(game)
                date = find_date(game)
                score = find_score(game)
                for team in teams:
                    save[-1].append(team)
                    if team not in clubs:
                        clubs.append(team)

                save[-1].append(score)
                for info in date:
                    save[-1].append(info)

            except UnicodeDecodeError:
                print('UnicodeDecodeError at ' + game)
                save.pop()
                
for competition in competitions2:
    years = sorted(glob(competition + '/*'))
    for year in years:
        games = sorted(glob(year + '/Game *'))
        for game in games:
            save.append([game[5:game.find('/', 5)]])
            try:
                teams = find_teams(game)
                if len(teams) == 2 and teams[0] in clubs and teams[1] in clubs:
                    date = find_date(game)
                    score = find_score(game)
                    for team in teams:
                        save[-1].append(team)
                        if team not in clubs:
                            clubs.append(team)

                    save[-1].append(score)
                    for info in date:
                        save[-1].append(info)
                else:
                    save.pop()

            except UnicodeDecodeError:
                # CdB - 2016 - game 122 is Fluminense vs Ypiranga
                print('UnicodeDecodeError at ' + game)
                save.pop()

No score found at Data/Serie A/2016/Game 378.csv, returned 0 x 0
UnicodeDecodeError at Data/Copa do Brasil/2016/Game 122.csv


In [5]:
for game in save[-5:]:
    print(game)

['Copa do Brasil', 'Bahia / BA', 'Atlético / MG', '2 x 1', '11', '2021', '04/08/2021']
['Copa do Brasil', 'Athlético / PR', 'Santos / SP', '1 x 0', '11', '2021', '25/08/2021']
['Copa do Brasil', 'Grêmio / RS', 'Flamengo / RJ', '0 x 4', '11', '2021', '25/08/2021']
['Copa do Brasil', 'São Paulo / SP', 'Fortaleza / CE', '2 x 2', '12', '2021', '25/08/2021']
['Copa do Brasil', 'Fluminense / RJ', 'Atlético / MG', '1 x 2', '12', '2021', '26/08/2021']


In [6]:
with open('data.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['Competition', 'Team 1', 'Team 2', 'Result', 'Round', 'Year', 'Date'])
    for game in save:
        writer.writerow(game)

Tendo realizado essa etapa, obtemos um arquivo ```csv``` contendo apenas os dados dos jogos de nosso interesse e, agora sim, vamos a Exploração dos Dados.

### Exploração dos Dados

Uma ideia inicial para essa etapa é visualizar nossos dados em um ```dataframe```, possibilitando uma visão geral dos mesmos.

In [7]:
df = pd.read_csv('data.csv')
df

,Competition,Team 1,Team 2,Result,Round,Year,Date
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN
...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021


Podemos ver que existem muitos dados faltantes na coluna Date. Isso nos dá duas opções:

+ excluir essa coluna e considerar as colunas Year e Round como lapso temporal (isso implicaria na exclusão das partidas da Copa do Brasil, que, como veremos abaixo, são 380 ao todo);
+ ou usar os valores que temos para inferir os valores faltantes (os jogos de uma mesma rodada tendem a ser no mesmo intervalo de 2 a 3 dias, o que facilitaria o processo).

In [8]:
count = 0
for game in save:
    if 'Copa do Brasil' in game[0]:
        count += 1
        
count

380

Nossa opção se dará pelo segundo caminho, ou seja, buscar inferir os valores faltantes por meio dos valores existentes. Para isso, vamos usar a interpolação pelos valores próximos. O código nas linhas 7 a 15 são para corrigir um problema de data, onde alguns valores estava sendo considerados no formato mm/dd/aaaa.

In [9]:
df['Date_by_Nearest'] = dates.date2num(pd.to_datetime(df['Date']))
df['Date_by_Nearest'].interpolate(method = 'nearest', inplace = True)
df['Date_by_Nearest'] = df['Date_by_Nearest'].apply(np.round)
df['Date_by_Nearest'] = dates.num2date(df['Date_by_Nearest'])
df['Date_by_Nearest'] = df['Date_by_Nearest'].dt.strftime('%d/%m/%Y')

date = list(df['Date_by_Nearest'])
for i in range(len(date)):
    if int(date[i][3:5]) > 12:
        date[i] = date[i][3:5] + '/' + date[i][:2] + '/' + date[i][-4:]
    else:
        if int(date[i][:2]) < 12:
            date[i] = date[i][3:5] + '/' + date[i][:2] + '/' + date[i][-4:]
            
df['Date_by_Nearest'] = date
df

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN,25/05/2013
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN,25/05/2013
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN,25/05/2013
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN,25/05/2013
...,...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021,25/08/2021


Mas note que isso gera algumas coisas estranhas, como a Série A de 2016, em que os jogos estão variando de junho de 2015 a maio de 2017 quando, na verdade, o campeonato ocorreu no período de [14 de maio de 2016 a 11 de dezembro de 2016](https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2016_-_S%C3%A9rie_A):

In [10]:
df.loc[1140:1519]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1140,Serie A,Palmeiras / SP,Athlético / PR,4 x 0,1,2016,NaN,12/06/2015
1141,Serie A,Flamengo / RJ,Sport / PE,1 x 0,1,2016,NaN,12/06/2015
1142,Serie A,Atlético / MG,Santos / SP,1 x 0,1,2016,NaN,12/06/2015
1143,Serie A,Coritiba / PR,Cruzeiro / MG,1 x 0,1,2016,NaN,12/06/2015
1144,Serie A,Botafogo / RJ,São Paulo / SP,0 x 1,1,2016,NaN,12/06/2015
...,...,...,...,...,...,...,...,...
1515,Serie A,Grêmio / RS,Botafogo / RJ,0 x 1,38,2016,NaN,14/05/2017
1516,Serie A,Athlético / PR,Flamengo / RJ,0 x 0,38,2016,NaN,14/05/2017
1517,Serie A,Chapecoense / SC,Atlético / MG,0 x 0,38,2016,NaN,14/05/2017
1518,Serie A,Ponte Preta / SP,Coritiba / PR,2 x 0,38,2016,NaN,14/05/2017


Para tentar driblar isso, vamos tentar uma abordagem um pouco diferente, considerando as datas existentes em uma rodada do ano desejado, quando disponíveis em grande quantidade, como, por exemplo, nas rodadas 34 e 35 de 2015:

In [11]:
df.loc[1090:1109]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1090,Serie A,Internacional / RS,Ponte Preta / SP,1 x 0,34,2015,07/11/2015,07/11/2015
1091,Serie A,Corinthians / SP,Coritiba / PR,2 x 1,34,2015,07/11/2015,07/11/2015
1092,Serie A,Athlético / PR,Avaí / SC,2 x 1,34,2015,07/11/2015,07/11/2015
1093,Serie A,Fluminense / RJ,Chapecoense / SC,2 x 3,34,2015,07/11/2015,07/11/2015
1094,Serie A,Palmeiras / SP,Vasco da Gama / RJ,0 x 2,34,2015,08/11/2015,08/11/2015
1095,Serie A,Cruzeiro / MG,São Paulo / SP,2 x 1,34,2015,08/11/2015,08/11/2015
1096,Serie A,Figueirense / SC,Atlético / MG,0 x 1,34,2015,NaN,08/11/2015
1097,Serie A,Flamengo / RJ,Goiás / GO,4 x 1,34,2015,NaN,08/11/2015
1098,Serie A,Sport / PE,Grêmio / RS,1 x 0,34,2015,08/11/2015,08/11/2015
1099,Serie A,Joinville / SC,Santos / SP,0 x 0,34,2015,08/11/2015,08/11/2015


Note que duas datas bem prováveis para os jogos 1096 e 1097 são os dias 07/11/2015 e 08/11/2015 (que foi o resultado da interpolação por proximidade). O mesmo ocorre para os jogos da rodada 35 (jogos 1101, 1103, 1107, 1108).

Por outro lado, as rodadas 1 e 2 de 2016, por exemplo, não podem ser inferidas por datas próximas, uma vez que nenhum dos jogos tem data. Nesses casos, vamos considerar as datas das respectivas rodadas em outros anos, isso é, os jogos da primeira rodada de 2017 foram realizados entre 13 e 15 de maio, logo, é esperado que a primeira rodada do campeonato de 2016 tenha sido realizada aproximadamente nessas datas também.

In [12]:
df.loc[1140:1149] # primeira rodada de 2016

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1140,Serie A,Palmeiras / SP,Athlético / PR,4 x 0,1,2016,NaN,12/06/2015
1141,Serie A,Flamengo / RJ,Sport / PE,1 x 0,1,2016,NaN,12/06/2015
1142,Serie A,Atlético / MG,Santos / SP,1 x 0,1,2016,NaN,12/06/2015
1143,Serie A,Coritiba / PR,Cruzeiro / MG,1 x 0,1,2016,NaN,12/06/2015
1144,Serie A,Botafogo / RJ,São Paulo / SP,0 x 1,1,2016,NaN,12/06/2015
1145,Serie A,Santa Cruz / PE,Vitória / BA,4 x 1,1,2016,NaN,12/06/2015
1146,Serie A,Corinthians / SP,Grêmio / RS,0 x 0,1,2016,NaN,12/06/2015
1147,Serie A,Figueirense / SC,Ponte Preta / SP,0 x 0,1,2016,NaN,12/06/2015
1148,Serie A,América / MG,Fluminense / RJ,0 x 1,1,2016,NaN,12/06/2015
1149,Serie A,Internacional / RS,Chapecoense / SC,0 x 0,1,2016,NaN,12/06/2015


In [13]:
df.loc[1520:1529] # primeira rodada de 2017

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1520,Serie A,Fluminense / RJ,Santos / SP,3 x 2,1,2017,14/05/2017,14/05/2017
1521,Serie A,Flamengo / RJ,Atlético / MG,1 x 1,1,2017,13/05/2017,13/05/2017
1522,Serie A,Palmeiras / SP,Vasco da Gama / RJ,4 x 0,1,2017,14/05/2017,14/05/2017
1523,Serie A,Corinthians / SP,Chapecoense / SC,1 x 1,1,2017,13/05/2017,13/05/2017
1524,Serie A,Cruzeiro / MG,São Paulo / SP,1 x 0,1,2017,14/05/2017,14/05/2017
1525,Serie A,Coritiba / PR,Atlético / GO,4 x 1,1,2017,15/05/2017,15/05/2017
1526,Serie A,Grêmio / RS,Botafogo / RJ,2 x 0,1,2017,14/05/2017,14/05/2017
1527,Serie A,Bahia / BA,Athlético / PR,6 x 2,1,2017,14/05/2017,14/05/2017
1528,Serie A,Ponte Preta / SP,Sport / PE,4 x 0,1,2017,14/05/2017,14/05/2017
1529,Serie A,Avaí / SC,Vitória / BA,0 x 0,1,2017,14/05/2017,14/05/2017


Vamos, então, ver os jogos sem data da Série A, em cada ano.

In [14]:
df.loc[0:3220] # serie A

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN,25/05/2013
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN,25/05/2013
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN,25/05/2013
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN,25/05/2013
...,...,...,...,...,...,...,...,...
3216,Serie A,Juventude / RS,São Paulo / SP,1 x 1,18,2021,29/08/2021,29/08/2021
3217,Serie A,Atlético / GO,Internacional / RS,0 x 0,18,2021,29/08/2021,29/08/2021
3218,Serie A,Bahia / BA,Fortaleza / CE,4 x 2,19,2021,04/09/2021,04/09/2021
3219,Serie A,Athlético / PR,Sport / PE,0 x 0,19,2021,05/09/2021,05/09/2021


In [15]:
fix = []
count = {}
for i in range(3221):
    if type(df.loc[i, 'Date']) != str:
        year = df.loc[i, 'Year']
        rd = df.loc[i, 'Round']
        fix.append([i, year, rd])
        if year in count:
            if rd in count[year]:
                count[year][rd] += 1
            else:
                count[year][rd] = 1
        
        else:
            count[year] = {}
            count[year][rd] = 1

print(len(fix))
count

923


{2013: {1: 8,
  2: 7,
  3: 9,
  4: 9,
  5: 10,
  6: 10,
  7: 7,
  8: 9,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 9,
  14: 10,
  15: 9,
  16: 10,
  17: 9,
  18: 10,
  19: 9,
  20: 10,
  21: 10,
  22: 10,
  23: 8,
  24: 10,
  25: 10,
  26: 9},
 2014: {38: 3},
 2015: {1: 10,
  2: 10,
  3: 10,
  4: 10,
  5: 10,
  6: 10,
  7: 10,
  8: 10,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 10,
  14: 10,
  15: 10,
  16: 10,
  17: 10,
  18: 10,
  19: 10,
  23: 7,
  24: 10,
  25: 9,
  26: 10,
  27: 10,
  28: 9,
  29: 9,
  30: 9,
  31: 10,
  32: 2,
  34: 2,
  35: 4,
  36: 3,
  37: 3,
  38: 3},
 2016: {1: 10,
  2: 10,
  3: 10,
  4: 10,
  5: 10,
  6: 10,
  7: 10,
  8: 10,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 10,
  14: 10,
  15: 10,
  16: 10,
  17: 10,
  18: 10,
  19: 10,
  20: 10,
  21: 10,
  22: 10,
  23: 10,
  24: 10,
  25: 10,
  26: 10,
  27: 10,
  28: 10,
  29: 10,
  30: 10,
  31: 10,
  32: 10,
  33: 9,
  34: 10,
  35: 10,
  36: 9,
  37: 10,
  38: 10},
 2017: {2: 1, 3: 4},
 2021: {1: 5}}

In [16]:
df.loc[3221:4031] # serie B

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
3221,Serie B,Palmeiras / SP,Atlético / GO,1 x 0,1,2013,25/05/2013,25/05/2013
3222,Serie B,América / MG,Figueirense / SC,2 x 4,2,2013,28/05/2013,28/05/2013
3223,Serie B,Atlético / GO,Joinville / SC,2 x 1,2,2013,NaN,28/05/2013
3224,Serie B,Figueirense / SC,Sport / PE,3 x 2,3,2013,NaN,01/06/2013
3225,Serie B,Palmeiras / SP,América / MG,0 x 1,3,2013,01/06/2013,01/06/2013
...,...,...,...,...,...,...,...,...
4027,Serie B,Avaí / SC,Coritiba / PR,1 x 2,20,2021,20/08/2021,20/08/2021
4028,Serie B,Vasco da Gama / RJ,Ponte Preta / SP,2 x 0,21,2021,29/08/2021,29/08/2021
4029,Serie B,Náutico / PE,Vitória / BA,1 x 1,21,2021,29/08/2021,29/08/2021
4030,Serie B,Coritiba / PR,Botafogo / RJ,0 x 1,21,2021,27/08/2021,27/08/2021


In [17]:
df.loc[4032:4070] # serie C

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
4032,Serie C,Santa Cruz / PE,Cuiabá / MT,1 x 0,3,2013,NaN,06/09/2021
4033,Serie C,Santa Cruz / PE,Fortaleza / CE,2 x 1,4,2013,13/07/2013,13/07/2013
4034,Serie C,Fortaleza / CE,Cuiabá / MT,3 x 0,8,2013,07/08/2013,07/08/2013
4035,Serie C,Cuiabá / MT,Santa Cruz / PE,1 x 3,12,2013,01/09/2013,01/09/2013
4036,Serie C,Fortaleza / CE,Santa Cruz / PE,2 x 0,13,2013,08/09/2013,08/09/2013
4037,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 1,17,2013,06/10/2013,06/10/2013
4038,Serie C,Fortaleza / CE,Cuiabá / MT,2 x 1,2,2014,NaN,06/10/2013
4039,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 1,11,2014,NaN,06/10/2013
4040,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 3,3,2015,NaN,06/10/2013
4041,Serie C,Juventude / RS,Portuguesa / SP,3 x 2,6,2015,NaN,06/10/2013


In [18]:
df.loc[4071:] # CdB

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
4071,Copa do Brasil,Santa Cruz / PE,Internacional / RS,0 x 0,9,2013,NaN,04/09/2021
4072,Copa do Brasil,Joinville / SC,Santos / SP,0 x 1,10,2013,08/05/2013,08/05/2013
4073,Copa do Brasil,Internacional / RS,Santa Cruz / PE,2 x 0,11,2013,NaN,08/05/2013
4074,Copa do Brasil,Avaí / SC,América / MG,0 x 3,11,2013,16/05/2013,16/05/2013
4075,Copa do Brasil,Santos / SP,Joinville / SC,0 x 0,12,2013,22/05/2013,22/05/2013
...,...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021,25/08/2021


Por fim, uma forma de analisar a coerência dos dados se dá pela análise da classificação final. Isso é, se a classificação final dos jogos com os dados coletados coincide com a classificação final do campeonato, os dados devem estar corretos (a probabilidade de ocorrerem dois ou mais erros, um anulando o outro, é muito baixa).

Para isso, tomamos a classificação final do campeonato:

In [19]:
year = 6 # 2013 = 0, 2014 = 1, etc
br = df.loc[380 * year:379 + 380 * year, ['Team 1', 'Team 2', 'Result']]
table = classification(br)
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
Flamengo / RJ,90,38,28,6,4,86,37,49
Santos / SP,74,38,22,8,8,60,33,27
Palmeiras / SP,74,38,21,11,6,61,32,29
Grêmio / RS,65,38,19,8,11,64,39,25
Athlético / PR,64,38,18,10,10,51,32,19
São Paulo / SP,63,38,17,12,9,39,30,9
Internacional / RS,57,38,16,9,13,44,39,5
Corinthians / SP,56,38,14,14,10,42,34,8
Fortaleza / CE,53,38,15,8,15,50,49,1
Goiás / GO,52,38,15,7,16,46,64,-18


Histograma de gols por partidas???

### Transformação dos Dados

escrever...

### Integração dos Dados

escrever...

### Formatação dos Dados

escrever...

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 